In [17]:
from PIL import Image
import numpy as np

class ImageGraph:
    class Node:
        def __init__(self, row, col, pix):
            self.row = row
            self.col = col
            self.ptr = self
            self.pix = pix
            self.int = 0
            self.size = 1

    def __init__(self, img_pth, k=1):
        # self.img_pth = img_pth
        self.img = img_pth
        # self.img = np.array(Image.open(img_pth).convert("L"), dtype=np.uint8)
        [self.rows, self.cols] = self.img.shape
        self.size = self.rows * self.cols
        self.nodes = [[self.Node(i, j, self.img[i, j]) for j in range(self.cols)] for i in range(self.rows)]
        self.k = k
        self.edges = []

    def segment(self):
        self.create_edges()
        self.edges.sort(key=lambda x: x[0])
        for edge in self.edges:
            weight = edge[0]
            n1 = edge[1][0]
            n2 = edge[1][1]
            c1 = self.find(n1)
            c2 = self.find(n2)
            if c1 == c2:
                continue
            mint = min(c1.int + self.t(c1.size), c2.int + self.t(c2.size))
            if weight <= mint:
                self.union(c1, c2, weight)

    def find(self, n):
        if n.ptr != n:
            n.ptr = self.find(n.ptr)
        return n.ptr

    def union(self, n1, n2, w):
        rootA, rootB = self.find(n1), self.find(n2)
        if rootA != rootB:
            rootA.ptr = rootB
            rootB.size += rootA.size
            rootB.int = w

    def w(self, i1, j1, i2, j2):
        pix1 = int(self.img[i1, j1])
        pix2 = int(self.img[i2, j2])
        return abs(pix1 - pix2)

    def create_edges(self, ):
        for i in range(self.rows):
            for j in range(self.cols):
                if i != self.rows-1:
                    self.edges.append([self.w(i, j, i+1, j), [self.nodes[i][j], self.nodes[i+1][j]]])
                if j != self.cols-1:
                    self.edges.append([self.w(i, j, i, j+1), [self.nodes[i][j], self.nodes[i][j+1]]])

    def t(self, size):
        threshold = self.k/size
        return threshold

    def color(self, i):
        B = i%256
        G = ((i-B)/256)%256
        R = ((i-B)/256**2)-G/256
        return np.array([R, B, G], dtype=np.uint8)

    def save(self, file_name):
        segmentation = np.zeros((self.rows, self.cols, 3))
        for i in range(self.rows):
            for j in range(self.cols):
                row = self.find(self.nodes[i][j]).row
                col = self.find(self.nodes[i][j]).col
                segmentation[i][j] = self.color(self.rows*row + col)
        segmentation = np.array(segmentation, dtype=np.uint8)
        # segmentation = Image.fromarray(segmentation)
        # segmentation.save(file_name)
        return segmentation

In [32]:
img = ImageGraph("chimp.png", 3000)
img.segment()
img.save("chimp-segment.png")

In [27]:
import numpy as np
import cv2
import imutils

cap = cv2.VideoCapture(0)

while(True):

    ret, frame = cap.read()
    
    frame = imutils.resize(frame, width=300)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = ImageGraph(gray, 1000)
    gray.segment()
    gray = gray.save("k")
    
    cv2.imshow('frame', gray)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()